In [1]:
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

c:\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create the agent
memory = InMemorySaver()
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [3]:
search = TavilySearch(max_results=2)
tools = [search]

In [4]:
llm_with_tools = model.bind_tools(tools)

sys_msg = SystemMessage(
    content="Use either tools to perform arithmetic on a set of inputs or just give me answer with your knowledge")

def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


In [5]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
builder.add_edge("assistant", END)


In [6]:
memory = InMemorySaver()
react_graph = builder.compile(checkpointer=memory)

# thread is identifies the session
config = {"configurable": {"thread_id": "1"}}

In [7]:
human_message = HumanMessage("Hi, I'm Srikanth and I live in Visakhapatnam.")
response = react_graph.invoke({"messages": [human_message]}, config=config)

In [8]:
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm Srikanth and I live in Visakhapatnam.
================================== Ai Message ==================================

Hello Srikanth from Visakhapatnam! It's nice to meet you. How can I help you today?


In [9]:
human_message = HumanMessage("Search for the weather where I live")
response = react_graph.invoke({"messages": [human_message]}, config=config)

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm Srikanth and I live in Visakhapatnam.
================================== Ai Message ==================================

Hello Srikanth from Visakhapatnam! It's nice to meet you. How can I help you today?
================================ Human Message =================================

Search for the weather where I live
================================== Ai Message ==================================
Tool Calls:
  tavily_search (d47eb1ae-cff5-49d6-954c-81875ac150e4)
 Call ID: d47eb1ae-cff5-49d6-954c-81875ac150e4
  Args:
    query: weather in Visakhapatnam
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Visakhapatnam", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://weather.com/weather/today/l/cd5651290b7018c92c71c869e464e3a12f73f295c1e02e54f25df8a304c7b22f", "title": "Visakhapatnam, A